# Inference 

In [1]:
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()

In [2]:
!set CV_IO_MAX_IMAGE_PIXELS=18500000000

In [3]:
import os 
from mmdet.apis import init_detector, show_result_pyplot
from mmrotate.apis import inference_detector_by_patches
import mmrotate
import torch
print(mmrotate)
mmrotate.__version__=='0.3.4'

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


<module 'mmrotate' from '/mnt/hdd/eric/.tmp_ipy/00.Reproduction_Test/mmrotate/__init__.py'>


True

In [4]:
choose_epoch = 9

checkpoint_file = f"/mnt/hdd/eric/.tmp_ipy/00.Checkpoint/ship_tmp/epoch_{choose_epoch}.pth"
config_file = '/mnt/hdd/eric/.tmp_ipy/00.Checkpoint/KARI_tmp/kari_fine_tunning.py'


In [5]:
from mmcv import Config

cfg = Config.fromfile(config_file)

In [6]:
#--------------------------------------
#----- classes
cfg.model.roi_head.bbox_head[0].num_classes=1 
cfg.model.roi_head.bbox_head[1].num_classes=1

classes = ('ship',)
cfg.data.train.classes=classes
cfg.data.val.classes=classes
cfg.data.test.classes=classes

# batch 
cfg.data.samples_per_gpu = 1

cfg.gpu_ids = [0]
cfg.device='cuda'
cfg.seed=22

# random seed
from mmdet.apis import set_random_seed
set_random_seed(cfg.seed,deterministic=True)

In [7]:
model = init_detector(cfg, checkpoint_file, device='cuda:0')

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: /mnt/hdd/eric/.tmp_ipy/00.Checkpoint/ship_tmp/epoch_9.pth
model to device #-----------------------------# :  cuda:0


In [8]:
from glob import glob 
tgt = []
for fold in os.listdir("/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO"):
    a_ = os.path.join( "/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO", fold)
    b_ = a_+"/*.png"
    files_ = glob(b_)
    tgt.append(files_)

In [9]:
tgt

[['/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/K3A_20230523045050_45040_00003611_L1G_PS/K3A_20230523045050_45040_00003611_L1G_PS_RGB.png'],
 ['/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/K3_20230328042158_57944_09401261_L1G_PS/K3_20230328042158_57944_09401261_L1G_PS_RGB.png'],
 ['/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/K3A_20230523045050_45040_00021531_L1G_PS/K3A_20230523045050_45040_00021531_L1G_PS_RGB.png'],
 ['/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/K3_20231028043354_61071_09341259_L1G_PS/K3_20231028043354_61071_09341259_L1G_PS_RGB.png'],
 ['/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/K3A_20231106050620_47570_00045344_L1G_PS/K3A_20231106050620_47570_00045344_L1G_PS_RGB.png'],
 ['/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/C1_20230430014955_11691_00006119_L1G_PS/C1_20230430014955_11691_00006119_L1G_PS_RGB.png'],
 ['/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/C1_20221227015155_09806_00006119_L1G/C1_20221227015155_09806_00006119_L1G_RGB

In [12]:
import cv2

#sample_png = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/VOC_Format/JPEGImages/100001615.bmp'
# whole_png = "/mnt/hdd/eric/.tmp_ipy/00.Reproduction_Test/black_sky_02_dirty.png"
#C1_20220420015713_05989_00121564_L1G_RGB
#C1_20221024015617_08833_00052297_L1G_RGB
#C1_20221103014737_08985_00027843_L1G_RGB
#C1_20221222015147_09730_00028974_L1G_RGB
#C1_20221227015155_09806_00006119_L1G_RGB
#whole_png = "/mnt/hdd/eric/.tmp_ipy/00.Data/ISRT/C1_20221227015155_09806_00006119_L1G_RGB.png"
#whole_png = "/mnt/hdd/eric/.tmp_ipy/00.Reproduction_Test/post_images/C1_20221227015155_09806_00006119_L1G_RGB_ver2.png"
#whole_png = "/mnt/hdd/eric/.tmp_ipy/00.Data/NTO/0-Pusan-2023-NTO/K3_20231113045054_61305_09361262_L1G_PS/K3_20231113045054_61305_09361262_L1G_PS_RGB.png"
pngs_ = [  "/mnt/hdd/eric/.tmp_ipy/00.Data/NTO/0-Pusan-2023-NTO/K3_20231115042854_61334_09361262_L1G_PS/K3_20231115042854_61334_09361262_L1G_PS_RGB.png"
          "/mnt/hdd/eric/.tmp_ipy/00.Data/NTO/0-Pusan-2023-NTO/K3A_20231107045100_47585_00043462_L1G_PS/K3A_20231107045100_47585_00043462_L1G_PS_RGB.png",
        "/mnt/hdd/eric/.tmp_ipy/00.Data/NTO/0-Pusan-2023-NTO/K3A_20231113045236_47676_00049406_L1G_PS/K3A_20231113045236_47676_00049406_L1G_PS_RGB.png"]

whole_png = pngs_[1]
a = cv2.imread(whole_png)

In [11]:
# for tg in tgt:
#     whole_png = tg[0]
import cv2

whole_png = "/mnt/hdd/eric/.tmp_ipy/00.Data/AO/0-Pusan-2023-AO/C1_20221227015155_09806_00006119_L1G/0-Pusan-2023-AO_1_C1_20221227015155_09806_00006119_L1G.png"
#print(whole_png)

from easydict import EasyDict

img = cv2.imread(whole_png)

args = EasyDict()
args.img = img
args.patch_sizes = [1024,800,2048]
args.patch_steps = [824,672,1920]
args.img_ratios = [1.0,1.5,2.0]

# default
#args.patch_sizes = [1024,800,2048]
#args.patch_steps = [824,672,1920]
#args.img_ratios = [1.0,1.5,2.0]

args.merge_iou_thr = 0.1
args.device = 'cuda:0'
args.score_thr = 0.5
args.palette = 'dota'


result = inference_detector_by_patches(model, args.img, args.patch_sizes,
                                        args.patch_steps, args.img_ratios,
                                        args.merge_iou_thr)
# show the results
show_result_pyplot(
    model,
    args.img,
    result,
    palette=args.palette,
    score_thr=args.score_thr,
    out_file=os.path.join("/mnt/hdd/eric/.tmp_ipy/00.Data/tmp_/one_class.png" ) )


#     out_file=os.path.join("/mnt/hdd/eric/.tmp_ipy/00.Reproduction_Test/post_images","result",[i for i in whole_png.split("/")][-1]) )

inference_detector_by_patches GPU ! :  cuda:0


/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


In [30]:
# Threshold 이상 선박

bboxes_ = [] 
cnt = 0
for r in result[0]:
    if r[-1] > 0.8:
        cnt+=1
        bboxes_.append(r)
#result[0][-1][-1]

In [32]:
# box numpy 로 저장 
import numpy as np

to_path = "/mnt/hdd/eric/.tmp_ipy/12.AIS_/2.Kari_Data"
n_name = [i for i in whole_png.split("/")][-1]

np.save(os.path.join(to_path,n_name),bboxes_)

In [31]:
bboxes_[0]

array([1.7207e+03, 2.8607e+03, 9.6675e+01, 1.7972e+01, 1.1884e+00, 9.9361e-01], dtype=float32)

In [21]:
bboxes_[1]

array([2.9710e+03, 5.9490e+03, 8.2574e+01, 1.8548e+01, 1.5191e+00, 9.6611e-01], dtype=float32)

In [22]:
# import os 

# # competition valid images 
# # competition dataset 의 위치 
# valid_image_path = "/mnt/hdd/eric/.tmp_ipy/00.Data/" + "oiltank_dataset/roas/test/images"

# test_images = [os.path.join(valid_image_path,i) for i in os.listdir(valid_image_path) if ".png" in i ]

# outputs= []
# cnt=0
# total_cnt = len(test_images)
# for img_name in test_images:
#     img = cv2.imread(f'{img_name}')
#     result =  inference_detector_by_patches(model,img, args.patch_sizes,
#                                         args.patch_steps, args.img_ratios,
#                                         args.merge_iou_thr)
#     outputs.append(result)
#     cnt+=1
#     print(cnt/total_cnt)

In [23]:
# outputs.__len__() # should be 68

In [24]:
# outputs[0][0][0]

In [25]:
# import numpy as np
# def center_to_corner(center_boxes):
#     center_boxes = torch.Tensor(center_boxes)
#     """
#     input
#     center_boxes : center coordinates boxes : [N, 4] (cx, cy, w, h)
#     output
#     corner_boxes : corner coordinates boxes : [N, 4] (x1, y1, x2, y2)
#     """
#     x1y1 = center_boxes[..., :2] - (center_boxes[..., 2:4])/2
#     x2y2 = center_boxes[..., :2] + (center_boxes[..., 2:4])/2
#     corner_boxes = torch.cat([x1y1, x2y2], dim=-1)
#     corner_boxes = np.array(corner_boxes)
#     return corner_boxes

In [26]:
# def conver4to8(box1 ): # this is corner coordinates ...
#     #xmin,ymin,w,h = box[0],box[1], box[2],box[3],
#     # 아,, faster RCNN은 corner output 이었는데,, 
#     # ,,,, ROI Transformer 는 또,, center format 이네 ㅋ
#     # 1 
#     x1 = box1[0]
#     y1 = box1[1]

#     x2 = box1[2]
#     y2 = box1[1]

#     x3 = box1[2]
#     y3 = box1[3]

#     x4 = box1[0]
#     y4 = box1[3]
#     return [x1,y1,x2,y2,x3,y3,x4,y4]


# Sub_ = {
#     'File':[],'Confidence':[],'X1':[],'Y1':[],'X2':[],'Y2':[],'X3':[],'Y3':[],'X4':[],'Y4':[],
#     }

# import os 
# f_names = [i[0:8] for i in os.listdir(valid_image_path)]

# for i,img in enumerate(outputs):
#     boxes = img[0]
#     filename = f_names[i]
#     for box in boxes:
#         #xmin,ymin,w,h,confi = box[0],box[1],box[2],box[3],box[4]
#         confi = box[5]
#         if confi >= 0.1:
#             corner_box = center_to_corner(box[0:4])
#             coords8 = conver4to8(corner_box)
#             confi = confi
            
#             Sub_['File'].append(filename)
#             Sub_['Confidence'].append(confi)
#             Sub_['X1'].append(coords8[0]) # 나중에 int 
#             Sub_['Y1'].append(coords8[1])
#             Sub_['X2'].append(coords8[2])
#             Sub_['Y2'].append(coords8[3])
#             Sub_['X3'].append(coords8[4])
#             Sub_['Y3'].append(coords8[5])
#             Sub_['X4'].append(coords8[6])
#             Sub_['Y4'].append(coords8[7])      

                

In [27]:
# import pandas as pd
# Sub_ = pd.DataFrame(Sub_)

In [28]:
# Sub_

In [29]:
# # 최종 submission 파일 저장위치 

# Sub_.to_csv("/mnt/hdd/eric/.tmp_ipy/00.Checkpoint/_재검증_final_submission_epoch_8.csv",index=False)